<a href="https://colab.research.google.com/github/ssawant/CNN/blob/main/Social_distance_detector_with_YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
import math

In [18]:
def is_close(p1, p2):
  """
  is_close: calcuate Euclidean distance between two point p1 and p2
  :param:
  p1, p2 - two point for calculating Euclidean distance

  :return:
  Euclidean distance between given two point
  """
  return math.sqrt(p1**2 + p2**2)

In [19]:
def convert_back(x, y, w, h):
  """
  convert_back: convert center coordinate to rectangle coordinate
  :param:
  x, y - midpoint of bbox
  w, h - width and height of bbox

  :return:
  xmin, ymin, xmax, ymax
  """

  xmin = int(round(x - (w/2)))
  xmax = int(round(x + (w/2)))
  ymin = int(round(y - (h/2)))
  ymax = int(round(y + (h/2)))

  return xmin, ymin, xmax, ymax

In [20]:
from itertools import combinations
import cv2

In [16]:
def cv_box(detections, img):
    """
    :param:
    detections - total detection in one frame
    img - image from detect_image method from darknet
  
    :return:
    img with bounding box
    """

    # Filtering the person class from detections and get bounding box centroid for each person detection

    if len(detections > 0):  # Atleast one detection in the image and check detection presence in the frame
        centroid_dict = dict()
        object_id = 0

        for detection in detections:  # if statement, we filter all the detections for person only
            # check for only person name tag
            name_tag = str(detector[0].decode())  # coco file have strings of all the name
            if name_tag == 'person':
                x, y, w, h = detection[2][0], detection[2][1], detection[2][2], detection[2][3]  # store center coordinate of the detection
                xmin, ymin, xmax, ymax = convert_back(float(x), float(y), float(w),
                                                      float(h))  # convert center coordinate to rectangle coordinate

                # Append center point of bbox for person detection.
                centroid_dict[object_id] = (int(x), int(y), xmin, ymin, xmax, ymax)
                object_id += 1  # increment object_id for each object(person) detected

        # Check which person bonding box are closer to each other

        red_zone_list = []  # List containing ObjectId under threshold distance condition.
        red_line_list = []

        for (id1, p1), (id2, p2) in combinations(centroid_dict.items(), 2):  # get all combination of close detection. List of multiple items - id1, p1 , p2, p3
            dx, dy = p1[0] - p2[0], p1[1] - p2[1]  # check distance between centroid x: 0, y: 1
            distance = is_close(dx, dy)  # calculates the Euclidean distance
            if distance < 75.0:  # set the social distance threashold
                if id1 not in red_zone_list:
                    red_zone_list.append(id1)  # add id to the red_zone list
                    red_line_list.append(p1[0:2])  # add points to the red_line list
                if id2 not in red_zone_list:
                    red_zone_list.append(id2)  # same for second id
                    red_line_list.append(p2[0:2])

        for idx, box in centroid_dict.items():  # dict (1(key):red(value), 2 blue) idx - key box - value
            if idx in red_zone_list:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (255, 0, 0), 2)  # Create red bounding box  #starting point, ending point size of 2
            else:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 255, 0), 2)  # Create green bounding box

        # Display risk analysis and risk indicator 

        text = f"No of at-risk people: {str(len(red_zone_list))}"  # Count people at risk
        location = (10, 25)  # Set location of display text
        cv2.putText(img, text, location, cv2.FONT_HERSHEY_SIMPLEX, 1, (246, 86, 86), 2, cv2.LINE_AA)  # Display Text

        for check in range(0, len(red_line_list) - 1):  # Draw line between nearby bbox iterate through red list items
            start_point = red_line_list[check]
            end_point = red_line_list[check + 1]
            check_line_x = abs(end_point[0] - start_point[0])  # Calculate the line coordinate for x
            check_line_y = abs(end_point[1] - start_point[1])  # Calculate the line coordinate for y
            if (check_line_x < 75) and (check_line_y < 25):
                cv2.line(img, start_point, end_point, (255, 0, 0), 2)  # Check the threshold and display the line

    return img

In [21]:
NET_MAIN = None
META_MAIN = None
ALT_MAIN = None

In [ ]:
!pip install darknet

In [33]:
import darknet

In [1]:
!nvidia-smi

Mon Mar 22 08:57:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# download config
!wget -O /content/yolov4.cfg "https://raw.githubusercontent.com/vincentgong7/VG_AlexeyAB_darknet/master/cfg/yolov4.cfg"
!wget -O /content/coco.data "https://raw.githubusercontent.com/vincentgong7/VG_AlexeyAB_darknet/master/cfg/coco.data"
!wget -O /content/yolov4.weights.7z.001 "https://github.com/vincentgong7/VG_AlexeyAB_darknet/raw/master/weights/yolov4.weights.7z.001"

In [ ]:
!7z x /content/yolov4.weights.7z.001

In [22]:
def yolo():
  """
  Perform object detection
  """
  global NET_MAIN, META_MAIN, ALT_MAIN
  config_path = ''
  weight_path = ''
  meta_path = ''

  



